Fase Dois: Modelagem Supervisionada
Modelo 1: Regressão Logística para Previsão de Inadimplência Futura
1. Objetivo do Modelo para a Persona de Negócios
Persona: Síndico, Administradora do Condomínio, Conselho Fiscal.

Objetivo: O modelo de Regressão Logística visa prever, para cada unidade, a probabilidade de ela não realizar o pagamento da sua cota condominial no próximo mês.

Benefício para a Persona: Com essa previsão, o síndico e a administradora podem antecipar ações preventivas. Isso inclui enviar lembretes personalizados, iniciar contatos para negociação antes do atraso se consolidar ou até mesmo priorizar unidades para acompanhamento mais próximo. O objetivo final é reduzir o índice de inadimplência e melhorar o fluxo de caixa do condomínio, evitando surpresas financeiras e a necessidade de rateios extras. A simplicidade e interpretabilidade da Regressão Logística a tornam adequada para explicar os resultados à gestão.

Para tornar o dataset mais completo do que na fase anterior, vamos acrescentar as colunas nome, idade, residentes(numero de pessoas no apartamento) e renda estimada associadas ao unidade_id com valores aproximados a realidade, mas preservando com nomes totalmente fictícios.

In [9]:
df.head(5)

,data,unidade_id,tipo_unidade,metragem,valor_cota,pagou,dias_atraso,valor_pago,despesa_total,obra_extraordinaria,pagou_num,media_pagamento_unidade
0,2020-01-01,Apt001,apartamento,39,556.45,True,0,556.45,33048.316687,0.0,1,0.950000
1,2020-01-01,Apt002,apartamento,55,619.22,True,0,619.22,33048.316687,0.0,1,0.983333
2,2020-01-01,Apt003,apartamento,39,568.89,True,0,568.89,33048.316687,0.0,1,0.950000
3,2020-01-01,Apt004,apartamento,39,572.01,True,0,572.01,33048.316687,0.0,1,0.966667
4,2020-01-01,Apt005,apartamento,39,575.91,False,30,243.17,33048.316687,0.0,0,0.133333


In [10]:
import pandas as pd
import numpy as np
from faker import Faker # Gerar nomes e dados fictícios

predio = pd.read_csv('condominio_atualizado.csv')

unique_units_info = predio[['unidade_id', 'tipo_unidade', 'metragem', 'media_pagamento_unidade']].drop_duplicates()

fake = Faker('pt_BR')
np.random.seed(42) 

unit_new_data = {}

for index, row in unique_units_info.iterrows():
    unit_id = row['unidade_id']
    tipo_unidade = row['tipo_unidade']
    metragem = row['metragem']
    media_pagamento = row['media_pagamento_unidade']

    #  NOME (Proprietário) 
    nome_proprietario = fake.name() 

    # IDADE (Proprietário) 
    idade_proprietario = np.random.randint(20, 80) # Idade entre 20 e 80 anos

    # RESIDENTES 
    num_residentes = 0
    if tipo_unidade == 'apartamento':
        base_res = np.random.randint(1, 3) # 1 ou 2 residentes base
        var_res_metragem = (metragem / 50) * np.random.rand() # Variação ligada à metragem
        num_residentes = int(base_res + var_res_metragem)
        num_residentes = max(1, min(num_residentes, 5)) # Mínimo 1, máximo 5
    else: # Lojas
        num_residentes = np.random.randint(0, 2) # 0 ou 1 (ex: dono ou funcionário)

    # RENDA_APROXIMADA 
    renda_aproximada = 0.0
    if tipo_unidade == 'apartamento':
        base_renda = np.random.lognormal(mean=np.log(5000), sigma=0.8) # Renda base para apartamentos
        ajuste_pagamento = (media_pagamento - 0.5) * 0.5 
        renda_aproximada = base_renda * (1 + ajuste_pagamento + (np.random.rand()-0.5)*0.2)
        renda_aproximada = max(1500.0, round(renda_aproximada, 2)) # Renda mínima de R$1500

    else: # Lojas
        base_renda = np.random.lognormal(mean=np.log(15000), sigma=1.2) # Renda base para lojas
        ajuste_pagamento = (media_pagamento - 0.5) * 0.2
        renda_aproximada = base_renda * (1 + ajuste_pagamento + (np.random.rand()-0.5)*0.1)
        renda_aproximada = max(3000.0, round(renda_aproximada, 2)) # Renda mínima de R$3000

    # Armazenar os dados para esta unidade
    unit_new_data[unit_id] = {
        'NOME': nome_proprietario,
        'IDADE': idade_proprietario,
        'RESIDENTES': num_residentes,
        'RENDA_APROXIMADA': renda_aproximada
    }

# CONVERSÃO dos dados coletados em um DataFrame
df_new_features_per_unit = pd.DataFrame.from_dict(unit_new_data, orient='index').reset_index().rename(columns={'index': 'unidade_id'})
predio = pd.merge(predio, df_new_features_per_unit, on='unidade_id', how='left')





# --- 9. Opcional: Salvar o DataFrame enriquecido para uso futuro ---
# predio.to_csv('condominio_enriquecido.csv', index=False)
# print("\nDataFrame enriquecido salvo como 'condominio_enriquecido.csv'")

In [20]:
predio.columns = predio.columns.str.upper()
predio.head(5)


,DATA,UNIDADE_ID,TIPO_UNIDADE,METRAGEM,VALOR_COTA,PAGOU,DIAS_ATRASO,VALOR_PAGO,DESPESA_TOTAL,OBRA_EXTRAORDINARIA,PAGOU_NUM,MEDIA_PAGAMENTO_UNIDADE,NOME,IDADE,RESIDENTES,RENDA_APROXIMADA
0,2020-01-01,Apt001,apartamento,39,556.45,True,0,556.45,33048.316687,0.0,1,0.950000,Jade Nascimento,58,2,9705.87
1,2020-01-01,Apt002,apartamento,55,619.22,True,0,619.22,33048.316687,0.0,1,0.983333,Diego Nascimento,38,1,22234.03
2,2020-01-01,Apt003,apartamento,39,568.89,True,0,568.89,33048.316687,0.0,1,0.950000,Bruna Jesus,55,2,3988.66
3,2020-01-01,Apt004,apartamento,39,572.01,True,0,572.01,33048.316687,0.0,1,0.966667,Bella Dias,40,1,9556.42
4,2020-01-01,Apt005,apartamento,39,575.91,False,30,243.17,33048.316687,0.0,0,0.133333,Carlos Eduardo Camargo,63,1,1723.58


In [8]:
import pandas as pd
# Replace 'your_data.csv' with the actual CSV filename you want to load
df = pd.read_csv('condominio_atualizado.csv')

In [7]:
df.head(5)

,data,unidade_id,tipo_unidade,metragem,valor_cota,pagou,dias_atraso,valor_pago,despesa_total,obra_extraordinaria,pagou_num,media_pagamento_unidade
0,2020-01-01,Apt001,apartamento,39,556.45,True,0,556.45,33048.316687,0.0,1,0.950000
1,2020-01-01,Apt002,apartamento,55,619.22,True,0,619.22,33048.316687,0.0,1,0.983333
2,2020-01-01,Apt003,apartamento,39,568.89,True,0,568.89,33048.316687,0.0,1,0.950000
3,2020-01-01,Apt004,apartamento,39,572.01,True,0,572.01,33048.316687,0.0,1,0.966667
4,2020-01-01,Apt005,apartamento,39,575.91,False,30,243.17,33048.316687,0.0,0,0.133333
